In [1]:
#detect1.py - treino
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten
from keras import optimizers
import numpy as np
import cv2, sys

2022-09-12 09:29:17.044206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-12 09:29:17.044237: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
if len(sys.argv)!=2:
    print("localize.py ent.png")
    sys.exit()

localize.py ent.png


SystemExit: 

/home/leonardo/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
num_classes = 10
qx0=cv2.imread(sys.argv[1],0);
nl, nc = qx0.shape[0], qx0.shape[1]
input_shape = (nl, nc, 1)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
model = Sequential(); #Entrada: 28*28*1
model.add(Conv2D(20, kernel_size=(5,5), activation='relu', input_shape=input_shape)) #Saida0:
20*24*24
model.add(MaxPooling2D(pool_size=(2,2))); #Saida1: 20*12*12
model.add(Conv2D(40, kernel_size=(5,5), activation='relu')); #Saida2: 40*8*8
model.add(MaxPooling2D(pool_size=(2,2))); #Saida3: 40*4*4
model.add(Conv2D(500, kernel_size=(4,4), activation='relu')) #Saida5: 500*1*1
model.add(Conv2D(100, kernel_size=(1,1), activation='relu')) #Saida6: 100*1*1
model.add(Conv2D(num_classes, kernel_size=(1,1), activation='softmax')) #Saida7: 10*1*1
model.load_weights("detect1.h5")

In [ ]:
qx0 = qx0.astype("float32")
qx0 /= 255 #0 a 1
qx0=qx0.reshape(1,nl,nc,1)
qp = model.predict(qx0)
print(qp.shape)

In [ ]:
snl, snc = qp.shape[1], qp.shape[2]
qp = qp.reshape(snl,snc,10)
sai=np.empty([snl,snc],dtype=np.int8)
for l in range(snl):
    for c in range(snc):
        i=np.argmax(qp[l,c,:])
        if qp[l,c,i]>0.999:
            #print(l,c,i,qp[l,c,i]);
            sai[l,c]=i;
        else:
            sai[l,c]=-1;
print(sai)
#print(qp)